In [1]:
from glob import glob
import os
import pandas as pd
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [15]:
root = "/hpcfs/users/a1232079/duyanh/MedSAM2/datasets/BTCV"
for subset in ["Tr", "Ts"]:
    samples = []
    for seg_name in os.listdir(os.path.join(root, f"labels{subset}")):
        if seg_name == ".":
            continue
        seg_path = os.path.join(root, f"labels{subset}", seg_name)
        seg = nib.load(seg_path)
        seg = np.asanyarray(seg.dataobj)
        for obj_id in np.unique(seg):
            if obj_id == 0:
                continue
            o_seg = (seg == obj_id)
            n_pos = np.argwhere(o_seg.sum(axis=(0,1))).shape[0]
            samples.append((os.path.join(f"labels{subset}", seg_name), obj_id, n_pos))
            
    df = pd.DataFrame(samples, columns=["gt_path", "obj_id", "n_pos"])
    df.to_csv(os.path.join("data/BTCV", f"labels{subset}.csv"))
    # print(df)